In [8]:
import sys
sys.path.insert(0,'C:\\Users\\Syahrir Ridha\\PycharmProjects\\NET_Solver\\')
import numpy as np
import torch
from geometry import *
from utils import Plot_Grid
from solver import *
from models import *
from mesh import *
from boundary import *
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import Dataset, DataLoader

In [9]:
# define the inputs
nx, ny = 10, 10
xi = np.linspace(0,1,nx)
eta = np.linspace(0,1,ny)
xx, yy = np.meshgrid(xi, eta)
xx, yy = xx.reshape(-1,1), yy.reshape(-1,1)
XI, ETA = torch.from_numpy(xx).float(), torch.from_numpy(yy).float()
inputs = torch.cat((XI,ETA),dim=1)

In [10]:
anulus = Analytical_Annulus(1., 0.6, 0.)
grid = TFI(XI, ETA, anulus)
x, y = grid.X(), grid.Y()

In [11]:
# define the neural network
in_dim = 2
out_dim = 1
layers = 5
hid_dim = 50

activation = Sine()
#activation = torch.nn.Tanh()

net = MLP(in_dim, out_dim, layers, hid_dim, act=activation)

In [12]:
class Trial_Solution:
    def __init__(self, model, nx, ny):
        self.model = model
        self.nx, self.ny = nx, ny
        
    def Dirichlet(self, model_data):
        # impose the dirichlet conditions
        # u = 0 @ eta 0 and 1
        model_data[0,:]=0
        model_data[-1,:] = 0
        
        return model_data
    
    def Neumann(self, model_data):
        # impose neumann boundary
        # dudy =0 @ xi = 0 and 1
        model_data[:,0] = model_data[:,1]
        model_data[:,-1] = model_data[:,-2]
        
        return model_data
    
    def __call__(self,inputs):
        # predict the general output
        pred = self.model(inputs)
        
        # reshape into a cartesian format
        pred = pred.reshape(self.nx, self.ny)
        
        # apply the boundary conditions
        # dirichlet
        out = self.Dirichlet(pred)
        
        # neumann 
        out = self.Neumann(out)
        
        # reshape back into original format
        out = out.reshape(-1,1)
        
        return out
    

In [21]:
class Physics_Loader(Dataset):
    def __init__(self,data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        return self.data[i]

In [22]:
trial = Trial_Solution(net, nx, ny)

In [26]:
load = Physics_Loader(net(inputs))

In [ ]:
class Loss:
    def __init__(self, trial, grid, model, nx, ny, inputs)
        self.trial = trial 
        self.grid = grid
        self.model = model
        self.nx, self.ny = nx, ny
        self.inputs = inputs
        
    def loss(self):
        xi, eta = self.inputs[:,0], self.inputs[:,1]
        x, y = self.grid.X(), self.grid.Y()
        
    
    
        
    def cal_grads(self, x, y, xi, eta):
        # compute all the gradients wrt x,y 
        dxdxi = self.compute_grad(x, xi)
        dxdeta = self.compute_grad(x, eta)
        dydxi = self.compute_grad(y, xi)
        dydeta = self.compute_grad(y, eta)
        jac = dxdxi*dydeta - dxdeta*dydxi
        
        return dxdxi, dxdeta, dydxi, dydeta, jac
    
    def compute_grad(self, outputs, inputs):
        gradient, = torch.autograd.grad(outputs, inputs, grad_outputs=outputs.data.new(outputs.shape).fill_(1),
                                        create_graph=True, retain_graph=True, only_inputs=True)
        return gradient